In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from pydantic import BaseModel

from pprint import pprint

In [2]:
# Set your API key
load_dotenv(dotenv_path='../secrets.env')
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

In [3]:
# sys_prompt = """ You are an expert in the stock market. You know all the companies whose stocks are listed on the respective stock exchanges and who are members of stock indexes such as the German DAX or the STOXX EUROPE 600 Index. You also know all the names or short forms these companies are known to the broader public. For instance: The German company and car manufacturer 'Bayerische Motoren Werke AG' is listed with this name in the German DAX índex but the company is better known as 'BMW' to the broader public. 
# Your task is to compile a list of tuples consisting of two strings each. The first string represents the official name of the company as listed on the stock exchange or in an equity stock index. The second string represents the short form of the company name such as 'BMW' in the example provided above. 
# Please be aware that short forms of company names do not exist not for all companies. If you cannot find a short form for a company name, leave the 'short_form' entry empty but fill in the 'name' entry for that company. Please make sure that all companies listed on the respective stock exchanges are included in your response even if this process takes quite some time. Please provide data only for those countries and stock indexes the user asks for. Please think step by step and also use all the tools and retrieval systems (such as internet searches) necessary to fully complete the task."""

In [4]:
sys_prompt = """You are an expert in the stock market. You know all the companies who are members European stock indices. You also know all the names or short forms these companies are known to the broader public. For instance: The German company and car manufacturer 'Bayerische Motoren Werke AG' is listed with this name in the German DAX índex but the company is better known as 'BMW' to the broader public. 
Your task is to compile a complete and full list of the stock indices the user asks for. The list is a list of tuples consisting of two strings each where the first string represents the official name of the company in the respective stock index and the second string represents the short form of the company name such as 'BMW' in the example provided above. Please make sure that all members of the respective stock index are included in your response even if this process takes quite some time. If you can not find a short form of a company name, only provide the 'name' of the company and leave the 'short_form' empty. Please think step by step and also use all the tools, retrieval systems (such as internet searches) or historical archives necessary to fully complete the task. The data does not have to be real time, it is sufficient if the data is from the previous or the current year. Please take your time, accuracy and completeness is more important than the duration of your response.
"""

In [5]:
user_prompt = """Please provide a comprehensive, thorough and complete list of all company names and their short forms (if there is one) for the following stock indices: STOXX EUROPE 600 Index, German DAX Index, French CAC 40 Index, Spanish IBEX 35 and Italian FTSE MIB Index. Make sure to avoid duplicates. """

In [6]:
class Step(BaseModel):
    explanation: str
    output: str

class Company(BaseModel):
    name: str
    short_form: str

class CompanyNamesAndShortForms(BaseModel):
    steps: list[Step]
    companies: list[Company]
    final_answer: str

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": user_prompt},
    ],
    response_format=CompanyNamesAndShortForms,
    # temperature=0.0
)

In [7]:
message = completion.choices[0].message

In [8]:
if message.parsed:
    pprint(message.parsed.steps)
    pprint(message.parsed.final_answer)
else:
    print(message.refusal)

[Step(explanation='Determine the companies listed in the selected European indices (STOXX EUROPE 600 Index, German DAX Index, French CAC 40 Index, Spanish IBEX 35, Italian FTSE MIB Index).', output='Identified the indices to research: German DAX, French CAC 40, Spanish IBEX 35, Italian FTSE MIB, and the broader STOXX EUROPE 600.'),
 Step(explanation='For each index, start by listing the companies. Use a reliable source to determine the companies within these indices. I will begin with the German DAX Index.', output='List for German DAX Index:\n1. Bayerische Motoren Werke AG (BMW)\n2. Daimler AG (Mercedes-Benz)\n3. Siemens AG (Siemens)\n4. Allianz SE (Allianz)\n5. BASF SE (BASF)\n6. Bayer AG (Bayer)\n7. SAP SE (SAP)\n8. Linde plc (Linde)\n9. Deutsche Telekom AG (Deutsche Telekom)\n10. Infineon Technologies AG (Infineon)\n11. Vonovia SE (Vonovia)\n12. Deutsche Post AG (Deutsche Post)\n13. Merck KGaA (Merck)\n14. Adidas AG (Adidas)\n15. Henkel AG & Co. KGaA (Henkel)\n16. E.ON SE (E.ON)\n1

In [9]:
if message.parsed:
    for comp in message.parsed.companies:
        print(f'comp: {comp.name} - {comp.short_form}')
else:
    print(message.refusal)

comp: Bayerische Motoren Werke AG - BMW
comp: Daimler AG - Mercedes-Benz
comp: Siemens AG - Siemens
comp: Allianz SE - Allianz
comp: BASF SE - BASF
comp: Bayer AG - Bayer
comp: SAP SE - SAP
comp: Linde plc - Linde
comp: Deutsche Telekom AG - Deutsche Telekom
comp: Infineon Technologies AG - Infineon
comp: Vonovia SE - Vonovia
comp: Deutsche Post AG - Deutsche Post
comp: Merck KGaA - Merck
comp: Adidas AG - Adidas
comp: Henkel AG & Co. KGaA - Henkel
comp: E.ON SE - E.ON
comp: RWE AG - RWE
comp: Deutsche Bank AG - Deutsche Bank
comp: MTU Aero Engines AG - MTU Aero Engines
comp: Fresenius SE & Co. KGaA - Fresenius
comp: Continental AG - Continental
comp: Beiersdorf AG - Beiersdorf
comp: HeidelbergCement AG - HeidelbergCement
comp: Siemens Healthineers AG - Siemens Healthineers
comp: Volkswagen AG - Volkswagen
comp: Delivery Hero SE - Delivery Hero
comp: HelloFresh SE - HelloFresh
comp: Sartorius AG - Sartorius
comp: Zalando SE - Zalando
comp: Symrise AG - Symrise
comp: LVMH Moët Hennessy 